This is my first experiment in this competition. Whereas XGBoost is highly recommended I rather tried to see how far I can go with an NN (using Keras).

This is the basic model and with 250 epochs has an accuracy of 80% (really poor).

I'll continue for a few days researching how much I can optimize this model.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir("../input/"))
print(os.listdir("../working/"))
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

['learn-together']
['.ipynb_checkpoints', '__notebook_source__.ipynb']
/kaggle/input/learn-together/test.csv
/kaggle/input/learn-together/train.csv
/kaggle/input/learn-together/sample_submission.csv


In [2]:
#load data, I had an issue with the data and hacked the data into the kernel
#dftrain=pd.read_csv('/kaggle/input/train.csv')
#dftest=pd.read_csv('/kaggle/input/test.csv')
dftrain=pd.read_csv('/kaggle/input/learn-together/train.csv')
dftest=pd.read_csv('/kaggle/input/learn-together/test.csv')

In [ ]:
#file shape
print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

In [ ]:
#features types
print(dftrain.dtypes)
print(dftest.dtypes)

In [ ]:
#validate files - nans per feature
print(dftrain.isnull().sum(axis = 0))
print(dftest.isnull().sum(axis = 0))

In [ ]:
#validate files - no rows with all zeros
print(dftrain[dftrain.drop(['Id','Cover_Type'], axis=1).eq(0).all(1)].empty)
print(dftest[dftest.drop('Id', axis=1).eq(0).all(1)].empty)

In [3]:
#split train data in features and labels
y = dftrain.Cover_Type
x = dftrain.drop(['Id','Cover_Type'], axis=1)

# split test data in features and Ids
Ids = dftest.Id
x_predict = dftest.drop('Id', axis=1)

In [4]:
#force all types to float
x = x.astype(float)
x_predict = x_predict.astype(float)

In [5]:
#normalize features
#in the future it can be done more elegantly, for now just using the max min values of the data that we know
#x['Elevation']=(x['Elevation']-x['Elevation'].min())/(x['Elevation'].max()-x['Elevation'].min())                             
x['Elevation']=(x['Elevation']-1859)/(3858-1859)                             
x['Aspect']=x['Aspect']/360                      
x['Slope']=x['Slope']/66                      
x['Horizontal_Distance_To_Hydrology']=x['Horizontal_Distance_To_Hydrology']/1397                      
x['Vertical_Distance_To_Hydrology']=(x['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x['Horizontal_Distance_To_Roadways']=x['Horizontal_Distance_To_Roadways']/7117                      
x['Hillshade_9am']=x['Hillshade_9am']/254                      
x['Hillshade_Noon']=x['Hillshade_Noon']/254                      
x['Hillshade_3pm']=x['Hillshade_3pm']/254                      
x['Horizontal_Distance_To_Fire_Points']=x['Horizontal_Distance_To_Fire_Points']/67173                      
                                
x_predict['Elevation']=(x_predict['Elevation']-1859)/(3858-1859)                             
x_predict['Aspect']=x_predict['Aspect']/360                      
x_predict['Slope']=x_predict['Slope']/66                      
x_predict['Horizontal_Distance_To_Hydrology']=x_predict['Horizontal_Distance_To_Hydrology']/1397                      
x_predict['Vertical_Distance_To_Hydrology']=(x_predict['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x_predict['Horizontal_Distance_To_Roadways']=x_predict['Horizontal_Distance_To_Roadways']/7117                      
x_predict['Hillshade_9am']=x_predict['Hillshade_9am']/254                      
x_predict['Hillshade_Noon']=x_predict['Hillshade_Noon']/254                      
x_predict['Hillshade_3pm']=x_predict['Hillshade_3pm']/254                      
x_predict['Horizontal_Distance_To_Fire_Points']=x_predict['Horizontal_Distance_To_Fire_Points']/67173                      

In [ ]:
#validate data - no rows with all zeros
#x.index[x.eq(0).all(1)]
print(x[x.eq(0).all(1)].empty)
print(x_predict[x_predict.eq(0).all(1)].empty)

In [6]:
# convert the label to One Hot Encoding
#to_categorical requires 0..6 instead of 1..7
y -=1
y = y.to_numpy()

num_classes = 7

from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [7]:
#convert the features dataframes to numpy arrays
x = x.to_numpy()
x_predict = x_predict.to_numpy()

In [8]:
#split in train (80%) and test (20%) sets 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [9]:
#here is the NN model
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

num_features = 54

model = Sequential()
model.add(Dense(units=num_features, activation='relu', kernel_initializer='normal', input_dim=num_features))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer='Adadelta',
              optimizer='Adam',
              metrics=['accuracy'])

Using TensorFlow backend.


In [10]:
#train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=250)

Train on 12096 samples, validate on 3024 samples
Epoch 1/250
12096/12096 [==============================] - 2s 126us/sample - loss: 1.1120 - acc: 0.5148 - val_loss: 0.9199 - val_acc: 0.6055
Epoch 2/250
12096/12096 [==============================] - 1s 96us/sample - loss: 0.8479 - acc: 0.6327 - val_loss: 0.8250 - val_acc: 0.6577
Epoch 3/250
12096/12096 [==============================] - 1s 97us/sample - loss: 0.7822 - acc: 0.6597 - val_loss: 0.8108 - val_acc: 0.6511
Epoch 4/250
12096/12096 [==============================] - 1s 98us/sample - loss: 0.7395 - acc: 0.6768 - val_loss: 0.7276 - val_acc: 0.6772
Epoch 5/250
12096/12096 [==============================] - 1s 98us/sample - loss: 0.7174 - acc: 0.6856 - val_loss: 0.7203 - val_acc: 0.6812
Epoch 6/250
12096/12096 [==============================] - 1s 98us/sample - loss: 0.7025 - acc: 0.6916 - val_loss: 0.7015 - val_acc: 0.6915
Epoch 7/250
12096/12096 [==============================] - 1s 98us/sample - loss: 0.6884 - acc: 0.7032 - val_l

12096/12096 [==============================] - 1s 96us/sample - loss: 0.3796 - acc: 0.8454 - val_loss: 0.5208 - val_acc: 0.8049
Epoch 59/250
12096/12096 [==============================] - 1s 99us/sample - loss: 0.3630 - acc: 0.8506 - val_loss: 0.5068 - val_acc: 0.8056
Epoch 60/250
12096/12096 [==============================] - 1s 99us/sample - loss: 0.3692 - acc: 0.8479 - val_loss: 0.4809 - val_acc: 0.8185
Epoch 61/250
12096/12096 [==============================] - 1s 96us/sample - loss: 0.3643 - acc: 0.8531 - val_loss: 0.5012 - val_acc: 0.8049
Epoch 62/250
12096/12096 [==============================] - 1s 96us/sample - loss: 0.3545 - acc: 0.8562 - val_loss: 0.5141 - val_acc: 0.8029
Epoch 63/250
12096/12096 [==============================] - 1s 95us/sample - loss: 0.3499 - acc: 0.8582 - val_loss: 0.4986 - val_acc: 0.8069
Epoch 64/250
12096/12096 [==============================] - 1s 101us/sample - loss: 0.3502 - acc: 0.8539 - val_loss: 0.5340 - val_acc: 0.7900
Epoch 65/250
12096/12096 

Epoch 116/250
12096/12096 [==============================] - 1s 106us/sample - loss: 0.2057 - acc: 0.9168 - val_loss: 0.5773 - val_acc: 0.8294
Epoch 117/250
12096/12096 [==============================] - 1s 106us/sample - loss: 0.2084 - acc: 0.9156 - val_loss: 0.5805 - val_acc: 0.8307
Epoch 118/250
12096/12096 [==============================] - 1s 109us/sample - loss: 0.2050 - acc: 0.9164 - val_loss: 0.5853 - val_acc: 0.8290
Epoch 119/250
12096/12096 [==============================] - 1s 96us/sample - loss: 0.2032 - acc: 0.9168 - val_loss: 0.6131 - val_acc: 0.8218
Epoch 120/250
12096/12096 [==============================] - 1s 106us/sample - loss: 0.2010 - acc: 0.9178 - val_loss: 0.6063 - val_acc: 0.8294
Epoch 121/250
12096/12096 [==============================] - 1s 101us/sample - loss: 0.2011 - acc: 0.9173 - val_loss: 0.5579 - val_acc: 0.8251
Epoch 122/250
12096/12096 [==============================] - 1s 105us/sample - loss: 0.2030 - acc: 0.9169 - val_loss: 0.6054 - val_acc: 0.8188


12096/12096 [==============================] - 1s 100us/sample - loss: 0.1300 - acc: 0.9478 - val_loss: 0.7367 - val_acc: 0.8320
Epoch 174/250
12096/12096 [==============================] - 1s 98us/sample - loss: 0.1270 - acc: 0.9488 - val_loss: 0.7041 - val_acc: 0.8370
Epoch 175/250
12096/12096 [==============================] - 1s 100us/sample - loss: 0.1303 - acc: 0.9486 - val_loss: 0.7231 - val_acc: 0.8356
Epoch 176/250
12096/12096 [==============================] - 1s 105us/sample - loss: 0.1236 - acc: 0.9496 - val_loss: 0.7575 - val_acc: 0.8350
Epoch 177/250
12096/12096 [==============================] - 1s 107us/sample - loss: 0.1363 - acc: 0.9454 - val_loss: 0.7801 - val_acc: 0.8194
Epoch 178/250
12096/12096 [==============================] - 1s 99us/sample - loss: 0.1349 - acc: 0.9474 - val_loss: 0.7546 - val_acc: 0.8320
Epoch 179/250
12096/12096 [==============================] - 1s 98us/sample - loss: 0.1297 - acc: 0.9503 - val_loss: 0.7376 - val_acc: 0.8330
Epoch 180/250
12

12096/12096 [==============================] - 1s 111us/sample - loss: 0.0818 - acc: 0.9683 - val_loss: 0.9085 - val_acc: 0.8254
Epoch 231/250
12096/12096 [==============================] - 1s 110us/sample - loss: 0.1061 - acc: 0.9609 - val_loss: 0.8988 - val_acc: 0.8370
Epoch 232/250
12096/12096 [==============================] - 1s 107us/sample - loss: 0.0828 - acc: 0.9674 - val_loss: 0.9100 - val_acc: 0.8370
Epoch 233/250
12096/12096 [==============================] - 1s 101us/sample - loss: 0.1086 - acc: 0.9563 - val_loss: 0.8849 - val_acc: 0.8380
Epoch 234/250
12096/12096 [==============================] - 1s 96us/sample - loss: 0.0811 - acc: 0.9679 - val_loss: 0.8260 - val_acc: 0.8446
Epoch 235/250
12096/12096 [==============================] - 1s 96us/sample - loss: 0.0978 - acc: 0.9630 - val_loss: 0.8608 - val_acc: 0.8419
Epoch 236/250
12096/12096 [==============================] - 1s 96us/sample - loss: 0.1022 - acc: 0.9614 - val_loss: 0.8818 - val_acc: 0.8323
Epoch 237/250
12

In [11]:
# Predict!!
y_predict = model.predict(x_predict)

In [12]:
for i in range(10):
	print(y_predict[i], np.argmax(y_predict[i])+1)

[8.414865e-01 7.237909e-02 4.306114e-05 2.715460e-08 6.398066e-05
 8.602745e-02 2.677694e-19] 1
[7.4224502e-01 2.5773385e-01 2.1245764e-13 2.0493008e-17 1.4452861e-05
 6.7365795e-06 7.5400352e-14] 1
[6.2340701e-01 3.7651679e-01 1.0213161e-09 1.8340593e-13 5.2895448e-05
 2.3341872e-05 4.0457043e-10] 1
[4.71093059e-01 5.28418005e-01 3.24236964e-07 9.71750198e-12
 3.89786524e-04 9.87956882e-05 1.23649615e-11] 2
[1.3467045e-01 8.6281317e-01 4.4106127e-06 4.8256316e-11 2.0503860e-03
 4.6154790e-04 2.5410815e-15] 2
[5.0568670e-01 4.9399433e-01 1.4308684e-07 6.5175539e-12 2.5272643e-04
 6.6201501e-05 8.6369453e-11] 1
[5.9468442e-01 4.0517539e-01 1.3997897e-08 1.3008982e-12 1.0347926e-04
 3.6679743e-05 3.0776687e-10] 1
[9.0184182e-01 9.8130278e-02 2.2322337e-11 1.4830036e-14 2.0826541e-05
 7.0796727e-06 2.5347999e-10] 1
[9.9848640e-01 1.5116740e-03 4.5566169e-11 8.5353376e-14 5.4590782e-07
 1.3468494e-06 1.2924902e-11] 1
[9.0687191e-01 9.3070656e-02 1.8019919e-09 5.0467210e-13 3.2119693e-05
 2

In [13]:
# Save predictions to a file for submission
#argmax give us the highest probable label
# we add one to the predictions to scale from 0..6 to 1..7
output = pd.DataFrame({'Id': Ids,
                       'Cover_Type': y_predict.argmax(axis=1)+1})
output.to_csv('submission.csv', index=False)

In [14]:
#create a link to download the file    
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv